# Método del "elemento pivote"

### Tabla de contenidos

1. Preámbulo: inicialización y muestra de la rdd
2. Rutinas:
    * 2.1 Matching
        * 2.1.1. Matching Directo
        * 2.1.2. Matching Complejo
    * 2.2. Clave/valor
    * 2.3. Compare and merge
    * 2.4. Rutina Global
3. Rutinas de Ejecucion y Display
    * 3.1. Ejecucion en simple
    * 3.2. Ejecucion en doble
4. Resultados con Matching Directo
    * 4.1. Resultados en simple
    * 4.2. Resultados en doble
5. Resultados con Matching Complejo
    * 5.1. Resultados en simple
    * 5.2. Resultados en doble


# Preámbulo

Spark context y lectura del archivo. 

In [1]:
from pyspark import SparkContext
sc=SparkContext()

ModuleNotFoundError: No module named 'pyspark'

In [2]:
rdd_derecho=sc.textFile('cociente_clean_uniform_od.csv').map(lambda x: x.split(';'))
rdd_derecho=rdd_derecho.map(lambda x: [x[0], x[1],x[2],x[3],eval(x[4])])
print('Número de registros: ', rdd_derecho.count())

Número de registros:  32152


In [3]:
rdd_izquierdo=sc.textFile('cociente_clean_uniform_oi.csv').map(lambda x: x.split(';'))
rdd_izquierdo=rdd_izquierdo.map(lambda x: [x[0], x[1],x[2],x[3],eval(x[4])])
print('Número de registros: ', rdd_izquierdo.count())

Número de registros:  34782


La RDD queda estructurada de la siguiente manera:
`[NUM,APELLIDOS,NOMBRE,FECHA,INDEX]`

Todos los campos son de tipo string

# Rutinas necesarias

### Función de matching directo

Compara por igualdad de campos

In [4]:
def match_directo(x,y,_):
    if x[0]==y[0]:
        if x[1]==y[1]:
            return True
        elif x[2]==y[2]:
            return True
    elif x[1]==y[1] and x[2]==y[2]:
        return True
    else:
        return False

### Matching compleja

Compara con fuzzy ratio (basada en levenshtein distance) en los campos de texto.

In [5]:
from fuzzywuzzy import fuzz

In [6]:
def match_complejo(x,y,npiv):
    if npiv==0:
        if x[2]==y[2]:
            if x[1]==y[1] or x[0]==y[0]:
                return True
            else:
                if fuzz.ratio(x[1],y[1])>90:
                    return True
                if fuzz.ratio(x[0],y[0])>90:
                    return True
            return False
        else:
            if fuzz.ratio(x[0],y[0])>90:
                if fuzz.ratio(x[1],y[1])>90:
                    return True
            return False
    if npiv==1 or npiv==2:
        if x[0]==y[0]:
            if x[2]==y[2]:
                return True
            if fuzz.ratio(x[1],y[1])>90:
                return True
        elif x[2]==y[2]:
            if fuzz.ratio(x[1],y[1])>90:
                return True
        return False
    if npiv==3:
        if x[0]==y[0]:
            if x[1]==y[1] or x[2]==y[2]:
                return True
            else:
                if fuzz.ratio(x[1],y[1])>90:
                    return True
                if fuzz.ratio(x[2],y[2])>90:
                    return True
            return False
        else:
            if fuzz.ratio(x[2],y[2])>90:
                if fuzz.ratio(x[1],y[1])>90:
                    return True
            return False

### Función de clave valor

Función que estructura la RDD sobre el elemento clave _(determinado por el valor de la variable global `npiv`)_, y se deja como valor el resto. El valor esta en forma de lista y no de tupla para poder editar sus valores al fusionar dos registros.

In [7]:
def clave_valor(x,npiv):
    clave=x[npiv]
    del x[npiv]
    return [clave,x]

def clave_valor_inversa(x,npiv):
    x[1].insert(npiv,x[0])
    return x[1]

### Compare and merge

Función que procede de la siguiente manera:

1. Si la entrada solo tiene un valor, lo devuelve y lo deja intacto.

2. En caso contrario, se transforma el conjunto de registros con la misma fecha a una lista `lista` para recorrerla de la siguiente manera: Se toma un elemento `i` de la lista (empezando **desde el final**), y se va comparando con el elemento primero, luego con el segundo, etc. 

3. En cuanto coincide con algun elemento `j`, se realizan las siguientes instrucciones:
    Primero, se añaden los índices asociados al paciente `i` a la lista de índices del paciente `j`.
    Segundo, si el numero de historia en la entrada `j` esta vacío (cosa bastante frecuente), se actualiza al de la entrada `i`.
    Por último, se borra la entrada `i` de la lista.

4. Se devuelve `[x[0],lista]`

In [8]:
def compare(x,npiv,funcion_matching):
    if len(x[1])==1:                          
        return (x[0],list(x[1]))
    else:                                      
        lista=list(x[1])
        for i in reversed(range(len(lista))):            
            for j in range(0,i):
                if funcion_matching(lista[i],lista[j],npiv):
                    lista=merge(lista,i,j)
                    break
        return (x[0],lista) 

In [9]:
def merge(lista,i,j):
    if len(lista[j][3])>len(lista[i][3]):
        lista[j][3]|=lista[i][3];
    elif len(lista[j][3])<len(lista[i][3]):
        lista[i][3]|=lista[j][3];
        lista[j]=lista[i];
    else:
        if lista[i][1]>=lista[j][1]:
            lista[j][1]=lista[i][1]
        if lista[i][0]>=lista[j][0]:
            lista[j][0]=lista[i][0]
        if lista[i][2]>=lista[j][2]:
            lista[j][1]=lista[i][1]
            
    del lista[i];
    return lista
    

## Rutina global

In [10]:
def pivote(rdd,npiv):
    rdd2=rdd.map(lambda x: clave_valor(x,npiv)).groupByKey()\
    .map(compare).flatMapValues(lambda x: x)\
    .map(lambda x: clave_valor_inversa(x,npiv))
    return rdd2

### Contamos el número de registros que no tienen número de historia

In [26]:
rdd_derecho.map(lambda x: (x[0])).filter(lambda x: x=='').count()

0

In [21]:
1161**2

1347921

In [11]:
def pivote_info(rdd,npiv,funcion_matching):
    nreg=rdd.count()
    t=time.time()
    
    rdd2=rdd.map(lambda x: clave_valor(x,npiv)).groupByKey()
    bloques=rdd2.count()
    comparaciones=rdd2.map(lambda x: len(list(x[1]))**2).sum()
    rdd2=rdd2.map(lambda x: compare(x,npiv,funcion_matching))\
    .flatMapValues(lambda x: x).map(lambda x: clave_valor_inversa(x,npiv))
    
    tiempo=time.time()-t
    nfinal=rdd2.count()
    duplicados=nreg-nfinal
    info=[round(tiempo,2),duplicados,nfinal,bloques,\
          round(nreg/bloques,2),comparaciones,\
          round(duplicados**2/comparaciones)]
    return rdd2, info

# Funciones de ejecución y display

In [12]:
import time
from pandas import DataFrame

## Rutinas para ejecución y display en simple

In [13]:
campos=["Numero", "Apellidos","Nombre","Fecha"]
info_headers=["Tiempo","Duplicados","Registros",\
              "Bloques","Registros/bloque","Comparaciones","EFICIENCIA"]

In [14]:
def resultados_ejecuciones_simple(rdd,funcion_matching):
    info=[[],[],[],[]]
    for i in range(4):
        _,info[i]=pivote_info(rdd,i,funcion_matching)

    data=DataFrame.from_records(info)
    data.columns=info_headers
    data.index=campos
    return data

## Rutinas para ejecución y display en doble

In [16]:
info_doble_headers=["Tiempo","Duplicados1", "Duplicados2",\
                    "Duplicados","Bloques","Comparaciones","EFICIENCIA"]

In [17]:
def pivote_doble_info(rdd,i,j,funcion_matching):
    rdd2,info1=pivote_info(rdd,i,funcion_matching)
    rdd2,info2=pivote_info(rdd2,j,funcion_matching)
    return rdd2,info1,info2

In [18]:
def resultados_ejecuciones_doble(rdd,funcion_matching):
    k=0
    info_doble=[[] for n in range(12)]
    campos_doble=["" for n in range(12)]
    for i in range(4):
        for j in range(4):
            if i!=j:
                _,info1,info2=pivote_doble_info(rdd,i,j,funcion_matching)
                info_doble[k]=[round(info1[0]+info2[0],2),\
                               info1[1],info2[1],info1[1]+info2[1],\
                               info1[3]+info2[3],info1[5]+info2[5],\
                               round((info1[1]+info2[1])**2/(info1[5]+info2[5]),2)]
                campos_doble[k]=campos[i]+"+"+campos[j]
                k+=1
    data=DataFrame(info_doble,columns=info_doble_headers,index=campos_doble)
    data=data.sort_values('EFICIENCIA',ascending=False)
    return data

# Resultados Matching Directo

## Resultados en simple

### Ojo derecho

In [19]:
directo_simple_derecho=resultados_ejecuciones_simple(rdd_derecho,match_directo)
directo_simple_derecho

,Tiempo,Duplicados,Registros,Bloques,Registros/bloque,Comparaciones,EFICIENCIA
Numero,1.19,6709,25443,23909,1.34,1399036,32
Apellidos,1.04,6546,25606,20839,1.54,115816,370
Nombre,0.87,7106,25046,4284,7.51,5165414,10
Fecha,0.88,5301,26851,15207,2.11,101838,276


### Ojo izquierdo

In [20]:
directo_simple_izquierdo=resultados_ejecuciones_simple(rdd_izquierdo,match_directo)
directo_simple_izquierdo

,Tiempo,Duplicados,Registros,Bloques,Registros/bloque,Comparaciones,EFICIENCIA
Numero,0.82,8098,26686,24840,1.40,1723346,38
Apellidos,0.84,8378,26406,21316,1.63,141018,498
Nombre,0.70,9070,25714,4493,7.74,6182064,13
Fecha,0.82,6422,28362,15616,2.23,118850,347


## Resultados doble

### Ojo derecho

In [21]:
directo_doble_derecho=resultados_ejecuciones_doble(rdd_derecho,match_directo)
directo_doble_derecho

,Tiempo,Duplicados1,Duplicados2,Duplicados,Bloques,Comparaciones,EFICIENCIA
Apellidos+Fecha,1.38,6546,1947,8493,35068,180450,399.73
Fecha+Apellidos,1.54,5301,3184,8485,34268,186225,386.60
Fecha+Numero,1.51,5301,3184,8485,37420,1249471,57.62
Apellidos+Numero,1.41,6546,1947,8493,43051,1286362,56.07
Numero+Fecha,1.67,6709,1744,8453,38140,1462621,48.85
Numero+Apellidos,1.55,6709,1744,8453,42988,1473251,48.50
Apellidos+Nombre,1.24,6546,1947,8493,24573,3459294,20.85
Fecha+Nombre,1.35,5301,3184,8485,18922,3872779,18.59
Numero+Nombre,1.49,6709,1744,8453,27646,4704589,15.19
Nombre+Fecha,1.56,7106,1395,8501,18505,5226612,13.83


### Ojo izquierdo

In [22]:
directo_doble_izquierdo=resultados_ejecuciones_doble(rdd_izquierdo,match_directo)
directo_doble_izquierdo

,Tiempo,Duplicados1,Duplicados2,Duplicados,Bloques,Comparaciones,EFICIENCIA
Apellidos+Fecha,1.38,8378,2199,10577,35653,210044,532.62
Fecha+Apellidos,1.63,6422,4121,10543,34931,215418,516.00
Fecha+Numero,1.71,6422,4121,10543,38089,1522314,73.02
Apellidos+Numero,1.43,8378,2199,10577,43789,1544054,72.45
Numero+Fecha,1.76,8098,2429,10527,39187,1794064,61.77
Numero+Apellidos,1.86,8098,2429,10527,44146,1807852,61.30
Apellidos+Nombre,1.37,8378,2199,10577,25141,3718240,30.09
Fecha+Nombre,1.69,6422,4121,10543,19429,4341870,25.60
Numero+Nombre,1.81,8098,2429,10527,28669,5413240,20.47
Nombre+Fecha,1.78,9070,1503,10573,18831,6246538,17.90


# Resultados Matching Complejo

### https://github.com/seatgeek/fuzzywuzzy

## Resultados simple

### Ojo derecho

In [23]:
complejo_simple_derecho=resultados_ejecuciones_simple(rdd_derecho,match_complejo)
complejo_simple_derecho

,Tiempo,Duplicados,Registros,Bloques,Registros/bloque,Comparaciones,EFICIENCIA
Numero,0.86,6916,25236,23909,1.34,1399036,34
Apellidos,0.89,6597,25555,20839,1.54,115816,376
Nombre,0.72,7333,24819,4284,7.51,5165414,10
Fecha,1.07,5541,26611,15207,2.11,101838,301


### Ojo izquierdo

In [24]:
complejo_simple_izquierdo=resultados_ejecuciones_simple(rdd_izquierdo,match_complejo)
complejo_simple_izquierdo

,Tiempo,Duplicados,Registros,Bloques,Registros/bloque,Comparaciones,EFICIENCIA
Numero,0.88,8432,26352,24840,1.40,1723346,41
Apellidos,0.95,8468,26316,21316,1.63,141018,508
Nombre,0.78,9424,25360,4493,7.74,6182064,14
Fecha,1.37,6752,28032,15616,2.23,118850,384


## Resultados doble

### Ojo derecho

In [25]:
complejo_doble_derecho=resultados_ejecuciones_doble(rdd_derecho,match_complejo)
complejo_doble_derecho

,Tiempo,Duplicados1,Duplicados2,Duplicados,Bloques,Comparaciones,EFICIENCIA
Apellidos+Fecha,1.28,6597,2151,8748,35058,180187,424.71
Fecha+Apellidos,1.31,5541,3195,8736,34076,185657,411.07
Fecha+Numero,1.48,5541,3295,8836,37338,1094467,71.34
Apellidos+Numero,1.38,6597,2130,8727,43037,1256549,60.61
Numero+Fecha,4.72,6916,1890,8806,38092,1461582,53.06
Numero+Apellidos,4.88,6916,1764,8680,42824,1472836,51.15
Apellidos+Nombre,1.20,6597,2163,8760,24540,3457963,22.19
Fecha+Nombre,1.27,5541,3275,8816,18864,3823727,20.33
Numero+Nombre,4.49,6916,1864,8780,27591,4668280,16.51
Nombre+Fecha,1.61,7333,1496,8829,18464,5225493,14.92


### Ojo izquierdo

In [26]:
complejo_doble_izquierdo=resultados_ejecuciones_doble(rdd_izquierdo,match_complejo)
complejo_doble_izquierdo

,Tiempo,Duplicados1,Duplicados2,Duplicados,Bloques,Comparaciones,EFICIENCIA
Apellidos+Fecha,1.31,8468,2472,10940,35639,209562,571.11
Fecha+Apellidos,1.53,6752,4161,10913,34665,214514,555.18
Fecha+Numero,1.82,6752,4338,11090,37953,1334746,92.14
Apellidos+Numero,1.30,8468,2480,10948,43762,1504340,79.68
Numero+Fecha,5.61,8432,2637,11069,39103,1792530,68.35
Numero+Apellidos,5.42,8432,2471,10903,43885,1807134,65.78
Apellidos+Nombre,1.28,8468,2532,11000,25074,3716004,32.56
Fecha+Nombre,1.32,6752,4291,11043,19340,4275946,28.52
Numero+Nombre,5.17,8432,2589,11021,28584,5349790,22.70
Nombre+Fecha,2.07,9424,1654,11078,18761,6244908,19.65
